In [1]:
from dm import Country

In [2]:
usa = Country('USA')

In [3]:
evars = usa.enrich_variables

len(evars.index)

9059

In [1]:
from setuptools import find_packages

In [4]:
find_packages('../../src', include=['dm', 'modeling'])

['dm', 'modeling']